In [3]:
from vllm import LLM, SamplingParams
from vllm.sampling_params import GuidedDecodingParams

from enum import Enum
from pydantic import BaseModel

In [4]:
MAX_TOKENS = 50

#### Guided decoding by Choice (list of possible options)

In [5]:
guided_decoding_params_choice = GuidedDecodingParams(choice=["Positive", "Negative"])
sampling_params_choice = SamplingParams(guided_decoding=guided_decoding_params_choice)

prompt_choice = "Classify this sentiment: vLLM is wonderful!"

#### Guided decoding by Regex

In [6]:
guided_decoding_params_regex = GuidedDecodingParams(regex=r"\w+@\w+\.com\n")
sampling_params_regex = SamplingParams(
    guided_decoding=guided_decoding_params_regex,
    stop=["\n"],
    max_tokens=MAX_TOKENS,
)

prompt_regex = (
    "Generate an email address for Alan Turing, who works in Enigma."
    "End in .com and new line. Example result:"
    "alan.turing@enigma.com\n"
)

#### Guided decoding by JSON using Pydantic schema

In [7]:
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

guided_decoding_params_json = GuidedDecodingParams(json=json_schema)
sampling_params_json = SamplingParams(
    guided_decoding=guided_decoding_params_json,
    max_tokens=MAX_TOKENS,
)

prompt_json = (
    "Generate a JSON with the brand, model and car_type of"
    "the most iconic car from the 90's"
)

#### Guided decoding by Grammar

In [8]:
simplified_sql_grammar = """
root ::= select_statement
select_statement ::= "SELECT " column " from " table " where " condition
column ::= "col_1 " | "col_2 "
table ::= "table_1 " | "table_2 "
condition ::= column "= " number
number ::= "1 " | "2 "
"""

guided_decoding_params_grammar = GuidedDecodingParams(grammar=simplified_sql_grammar)
sampling_params_grammar = SamplingParams(
    guided_decoding=guided_decoding_params_grammar,
    max_tokens=MAX_TOKENS,
)

prompt_grammar = (
    "Generate an SQL query to show the 'username' and 'email'from the 'users' table."
)

### Generation

In [9]:
def format_output(title: str, output: str):
    print(f"{'-' * 50}\n{title}: {output}\n{'-' * 50}")

def generate_output(prompt: str, sampling_params: SamplingParams, llm: LLM):
    outputs = llm.generate(prompt, sampling_params=sampling_params)
    return outputs[0].outputs[0].text

In [10]:
def main():
    llm = LLM(model="Qwen/Qwen2.5-3B-Instruct", max_model_len=100)

    choice_output = generate_output(prompt_choice, sampling_params_choice, llm)
    format_output("Guided decoding by Choice", choice_output)

    regex_output = generate_output(prompt_regex, sampling_params_regex, llm)
    format_output("Guided decoding by Regex", regex_output)

    json_output = generate_output(prompt_json, sampling_params_json, llm)
    format_output("Guided decoding by JSON", json_output)

    grammar_output = generate_output(prompt_grammar, sampling_params_grammar, llm)
    format_output("Guided decoding by Grammar", grammar_output)

In [11]:
main()

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

INFO 09-05 21:13:52 [config.py:853] This model supports multiple tasks: {'embed', 'score', 'generate', 'reward', 'classify'}. Defaulting to 'generate'.


tokenizer_config.json: 0.00B [00:00, ?B/s]

INFO 09-05 21:13:52 [config.py:1467] Using max model len 100
INFO 09-05 21:13:59 [config.py:2267] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 09-05 21:13:59 [config.py:4566] full_cuda_graph is not supported with cascade attention. Disabling cascade attention.


vocab.json: 0.00B [00:00, ?B/s]

...

--------------------------------------------------
Guided decoding by JSON: {"brand": "Lamborghini", "model": "Diablo", "car_type": "Coupe"}
--------------------------------------------------


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0% 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

--------------------------------------------------
Guided decoding by Grammar: SELECT col_1  from table_1  where col_1 = 1 
--------------------------------------------------


[rank0]:[W905 21:15:23.143405678 ProcessGroupNCCL.cpp:1476] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())
